In [1]:
# Midterm Q: How does a segregation score help determine if a congressional district is racially gerrymandered? 
# How can we visualize this? 
# Data from Sal ty Sal.

# We would like the user to be able to select from VTDs and create their own legeslative district in the state of Virginia.
# They would reicieve stats on the population, compactness, and S+/- score of their disctrict.
import pandas as pd

This link takes you to a preliminary interface where users can select their districts

https://wm-gis.maps.arcgis.com/home/item.html?id=1938f79f1a9b4bac9c50fff09d17b662


In [2]:
# The following is an example of calculating S+/- scores
 
# Get the column names bc I did not copy them over from original data to state data csv
# It does not include block_geoid_end, but thats not a necessary column. 
header = pd.read_csv('/Users/rebeccawagner/Documents/GitHub/GerryGainMCMC/local_environments_US.csv', index_col=0, nrows=0).columns.tolist()

# Get the data for local environments of VA census blocks 
# For some states with FIPS that have leading 0s, we need the dtype parameter. Not here, but for the future I left it.
# Reset the index to overwrite block_geoid_end with regular indecies
VA_data = pd.read_csv('/Users/rebeccawagner/Documents/GitHub/GerryGainMCMC/state_environs/VA_LE', names = header, dtype={'block_geoid_start':str}).reset_index(drop=True)

In [3]:
VA_data

,block_geoid_start,sl_dist,rn_total_sl_dist,rn_hisp_sl_dist,rn_nh_white_sl_dist,rn_nh_black_sl_dist,state_fips,geoid20,geoid,total,...,hisp,nh_white,nh_black,nh_na,nh_asn,nh_haw,nh_oth,nh_multi,block_id_crosswalk,_merge
0,510010901011004,87.167068,784570,66249,436099,206975,51,510010901011004,510010901011004,11,...,0,7,1,0,1,0,0,2,2,matched (3)
1,510010901011006,89.001945,784639,66463,436571,206179,51,510010901011006,510010901011006,48,...,2,44,0,1,0,0,0,1,3,matched (3)
2,510010901011011,87.635887,784723,66369,436424,206518,51,510010901011011,510010901011011,13,...,0,9,0,0,0,0,0,4,4,matched (3)
3,510010901011012,87.547806,784683,66456,436459,206349,51,510010901011012,510010901011012,64,...,0,64,0,0,0,0,0,0,5,matched (3)
4,510010901011019,86.464142,784607,66923,435224,206419,51,510010901011019,510010901011019,4,...,0,4,0,0,0,0,0,0,6,matched (3)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117355,518400003024016,43.091900,784675,90374,494402,52580,51,518400003024016,518400003024016,113,...,89,18,1,0,0,0,0,5,117357,matched (3)
117356,518400003024019,42.706642,784708,89072,497661,52252,51,518400003024019,518400003024019,209,...,88,82,29,0,0,1,0,9,117358,matched (3)
117357,518400003024020,42.850006,784720,89008,498184,52093,51,518400003024020,518400003024020,138,...,88,32,11,0,0,0,0,7,117359,matched (3)
117358,518400003024022,42.632431,783929,89099,496172,52300,51,518400003024022,518400003024022,13,...,8,0,0,0,3,0,0,2,117360,matched (3)


In [4]:
# We only need these columns to create a S+/- score:
    # block_geoid_start = census block
    # rn_total_sl_dist = population of block's local environment
    # rn_nh_black_sl_dist = black population of block's local environment
    # total = population of block
    # nh_black = black population of block 

VA_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117360 entries, 0 to 117359
Data columns (total 22 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   block_geoid_start    117360 non-null  object 
 1   sl_dist              117360 non-null  float64
 2   rn_total_sl_dist     117360 non-null  int64  
 3   rn_hisp_sl_dist      117360 non-null  int64  
 4   rn_nh_white_sl_dist  117360 non-null  int64  
 5   rn_nh_black_sl_dist  117360 non-null  int64  
 6   state_fips           117360 non-null  int64  
 7   geoid20              117360 non-null  int64  
 8   geoid                117360 non-null  int64  
 9   total                117360 non-null  int64  
 10  lat                  117360 non-null  float64
 11  lon                  117360 non-null  float64
 12  hisp                 117360 non-null  int64  
 13  nh_white             117360 non-null  int64  
 14  nh_black             117360 non-null  int64  
 15  nh_na            

In [5]:
# Subset the columns we need
VA_data_score = VA_data[['block_geoid_start', 'rn_total_sl_dist', 'rn_nh_black_sl_dist', 'total', 'nh_black']]
VA_data_score

,block_geoid_start,rn_total_sl_dist,rn_nh_black_sl_dist,total,nh_black
0,510010901011004,784570,206975,11,1
1,510010901011006,784639,206179,48,0
2,510010901011011,784723,206518,13,0
3,510010901011012,784683,206349,64,0
4,510010901011019,784607,206419,4,0
...,...,...,...,...,...
117355,518400003024016,784675,52580,113,1
117356,518400003024019,784708,52252,209,29
117357,518400003024020,784720,52093,138,11
117358,518400003024022,783929,52300,13,0


In [6]:
# Create the percentages we need

# Calculate the percentage of racial group m (black) in the local environment of the census block
# running total black / running total 
VA_data_score['pct_m_le'] = VA_data_score['rn_nh_black_sl_dist']/VA_data_score['rn_total_sl_dist']*100

# Weight this percent by the number of people in the census block
# weighted_m_le * total
VA_data_score['weighted_pct_m_le'] = VA_data_score['pct_m_le'] * VA_data_score['total']

# Calculate the percent of racial group m (black) in each block
VA_data_score['pct_black_block'] = VA_data_score['nh_black']/VA_data_score['total']*100

VA_data_score

/tmp/ipykernel_24417/352572142.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  VA_data_score['pct_m_le'] = VA_data_score['rn_nh_black_sl_dist']/VA_data_score['rn_total_sl_dist']*100
/tmp/ipykernel_24417/352572142.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  VA_data_score['weighted_pct_m_le'] = VA_data_score['pct_m_le'] * VA_data_score['total']
/tmp/ipykernel_24417/352572142.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

,block_geoid_start,rn_total_sl_dist,rn_nh_black_sl_dist,total,nh_black,pct_m_le,weighted_pct_m_le,pct_black_block
0,510010901011004,784570,206975,11,1,26.380693,290.187619,9.090909
1,510010901011006,784639,206179,48,0,26.276925,1261.292391,0.000000
2,510010901011011,784723,206518,13,0,26.317312,342.125056,0.000000
3,510010901011012,784683,206349,64,0,26.297116,1683.015434,0.000000
4,510010901011019,784607,206419,4,0,26.308585,105.234340,0.000000
...,...,...,...,...,...,...,...,...
117355,518400003024016,784675,52580,113,1,6.700863,757.197566,0.884956
117356,518400003024019,784708,52252,209,29,6.658783,1391.685570,13.875598
117357,518400003024020,784720,52093,138,11,6.638419,916.101794,7.971014
117358,518400003024022,783929,52300,13,0,6.671523,86.729793,0.000000


In [7]:
# Now we have all the data points we need to create S+/- scores for any combination of census blocks. In our case, we would like to use VTDs to create S+/- scores, so we will need to know which blocks are in each VTD and aggregate that data. 

# Let's run a toy example with the first 20 census blocks in VA. 
toy_example = VA_data_score.head(20)

# Pretend the first 10 blocks are in VTD 1 and the next 10 blocks are in VTD 2. Add this column for reference.
example_VTDs = [1] * 10 + [2] * 10
toy_example.insert(1,'VTD',example_VTDs)

toy_example

,block_geoid_start,VTD,rn_total_sl_dist,rn_nh_black_sl_dist,total,nh_black,pct_m_le,weighted_pct_m_le,pct_black_block
0,510010901011004,1,784570,206975,11,1,26.380693,290.187619,9.090909
1,510010901011006,1,784639,206179,48,0,26.276925,1261.292391,0.000000
2,510010901011011,1,784723,206518,13,0,26.317312,342.125056,0.000000
3,510010901011012,1,784683,206349,64,0,26.297116,1683.015434,0.000000
4,510010901011019,1,784607,206419,4,0,26.308585,105.234340,0.000000
5,510010901011020,1,785065,206606,18,0,26.317057,473.707018,0.000000
6,510010901011021,1,784731,206404,33,0,26.302516,867.983041,0.000000
7,510010901011022,1,784697,206412,7,0,26.304676,184.132729,0.000000
8,510010901011024,1,785067,207099,57,5,26.379787,1503.647842,8.771930
9,510010901011025,1,784889,207033,17,0,26.377360,448.415126,0.000000


In [8]:
# Total population in VTD 1
VTD_1_total = toy_example.loc[toy_example['VTD']==1,'total'].sum()
# Total black population in VTD 1
VTD_1_black_total = toy_example.loc[toy_example['VTD']==1,'nh_black'].sum()
# Total weighted black population in LE of VTD 1 
VTD_1_weighted_pct_m_le = toy_example.loc[toy_example['VTD']==1, 'weighted_pct_m_le'].sum()
# Percent of group m (black) in VTD 1
VTD_1_pct_black_block = (VTD_1_black_total/VTD_1_total) * 100

# Total population in VTD 2
VTD_2_total = toy_example.loc[toy_example['VTD']==2,'total'].sum()
# Total black population in VTD 2
VTD_2_black_total = toy_example.loc[toy_example['VTD']==2,'nh_black'].sum()
# Total weighted black population in LE of VTD 2
VTD_2_weighted_pct_m_le = toy_example.loc[toy_example['VTD']==2, 'weighted_pct_m_le'].sum()
# Percent of group m (black) in VTD 2
VTD_2_pct_black_block = (VTD_2_black_total/VTD_2_total) * 100

In [9]:
# Calculate S+/- for just VTD 1
# Divide sum of weighted percent population m in LE by total population in district 
# and subtract from percentage of people in districct who are members of group m
VTD_1_score = VTD_1_pct_black_block - (VTD_1_weighted_pct_m_le/VTD_1_total)

# Calculate S+/- for just VTD 2
VTD_2_score = VTD_2_pct_black_block - (VTD_2_weighted_pct_m_le/VTD_2_total)

(VTD_1_score, VTD_2_score)

(-24.11669336469764, -24.654198231318816)

In [10]:
# Now say we select both VTDs. What is the S+/- score then?

# Total population in district
district_total = VTD_1_total + VTD_2_total
# Total black population in district 
district_black_total = VTD_1_black_total + VTD_2_black_total
# Total weighted black population in LE of district
district_weighted_pct_m_le = VTD_1_weighted_pct_m_le + VTD_2_weighted_pct_m_le
# Percent of group m (black) in district
district_pct_black_block = (district_black_total/district_total) * 100

# Cacoluate S+/-
district_score = district_pct_black_block - (district_weighted_pct_m_le/district_total)

district_score

-24.39451559593523

In [11]:
# So to calculate S+/- scores in time with a user selecting VTDs, we will need to know these three stats for each VTD:
    # the total population of the VTD
    # the total minority population of the VTD
    # The total (summed) weighted percent of minority population in local evironments 
# From there we sum these stats for each VTD and calculate the minority percent in the district
# Subtract the district weighted percent / district total from the minority district population